## Retrieval Augmented Generation or RAG


#### 1. Import the Environment Variables


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

#### 2. Load & Split the Data (Chunking)


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load the document
loader = TextLoader("dsa.md")
docs = loader.load()

In [3]:
# 2. Split into chunks
# chunk_size=100: We want small chunks for this simple example
# chunk_overlap=20: Keeps context between chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

print(f"✅ Data split into {len(splits)} chunks.")
print("Example chunk:", splits[0].page_content)

✅ Data split into 24 chunks.
Example chunk: # **1. Introduction to Data Structures**

### **Theory**

A **data structure** is a method of organizing data in memory so that it can be efficiently accessed and modified.
Operations like searching, insertion, deletion, traversal depend on how data is organized.

Examples:
Arrays, Linked Lists, Trees, Graphs, Hash Tables, etc.

---

# **2. Arrays**

### **Theory**


#### 3. Embeddings & Vector Store


In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create the Vector Store (This stores our data as vectors)
vector_store = Chroma(
    embedding_function=embeddings,
    collection_name="dsa_data",  # Name your collection
    persist_directory="chroma_db",
)

# Add the documents to the Vector store
vector_store.add_documents(splits)

print("✅ Vector Store created successfully!")

✅ Vector Store created successfully!


#### 4. The Retriever


In [5]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Let's test the retrieval before connecting the LLM
test_query = "What is Sorting?"
results = retriever.invoke(test_query)

print(f"🔍 Test Retrieval for query: '{test_query}'")
print(f"📄 Found {len(results)} relevant chunks.")

🔍 Test Retrieval for query: 'What is Sorting?'
📄 Found 5 relevant chunks.


In [6]:
for index, result in enumerate(results):
    print("=" * 20, index + 1, "=" * 20)
    print(result.page_content)

==================== 1 ====================
# Delete
del hash_table["age"]
```

---

# **10. Sorting Algorithms**

---

## **Bubble Sort**

### **Theory**

Repeatedly compares adjacent elements and swaps them.

-   Time: **O(n²)**
-   Space: **O(1)**
    Simple but inefficient.

### **Code**

```python
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

print(bubble_sort([5, 1, 4, 2]))
```
==================== 2 ====================
print(bubble_sort([5, 1, 4, 2]))
```

---

## **Merge Sort**

### **Theory**

A **divide-and-conquer** algorithm:

1. Split
2. Sort recursively
3. Merge

-   Time: **O(n log n)**
-   Space: **O(n)**

### **Code**

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr)//2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])

    i = j = 0
    merged = []
==========

#### 5. The RAG Chain (Generation)


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. Setup the LLM (The "Brain")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 2. Define the System Prompt
# This tells the AI: "You are an assistant. Use the context I give you."
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [8]:
from langchain_text_splitters.base import Document


def docs_to_string(docs: list[Document]) -> str:
    docs_str = "\n\n".join([doc.page_content for doc in docs])
    return docs_str

In [9]:
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field


class CodeModel(BaseModel):
    theory: str = Field(description="Provide the description")
    code: str = Field(description="Provide the code")


rag_chain = (
    {
        "context": retriever | docs_to_string,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm.with_structured_output(CodeModel)
)

/Users/yashjain/Developer/youtube/langchain-youtube/.langchain-youtube-env/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:2041: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [16]:
response: CodeModel = rag_chain.invoke("What is DFS, also provide the code?")

In [17]:
print(response.theory)

Depth First Search (DFS) is a graph traversal algorithm that explores as far as possible along each branch before backtracking. It uses a stack to keep track of the nodes to visit next.


In [18]:
print(response.code)

def dfs(node, visited=set()):
    if node not in visited:
        print(node, end=" ")
        visited.add(node)
        for neighbor in graph[node]:
            dfs(neighbor, visited)

dfs(1)
